## Create AnnData Object for Analysis Pipeline
#### Anna Möller anna.moeller@fau.de

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import json
import os
import cv2
import time
import numpy as np
from csbdeep.utils import Path, normalize
from segmentation import MELC_Segmentation
import pandas as pd
import anndata as ad
import pickle
import sys
sys.path.append("/data_slow/je30bery/spatial_proteomics/marker_expression/")
from initial_analysis import ExpressionAnalyzer
import anndata as ad

2023-07-26 13:45:46.908395: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 13:45:46.909469: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 13:45:46.933712: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 13:45:46.934136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 13:45:47.342488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
with open("/data_slow/je30bery/spatial_proteomics/segmentation/anndata_example.pkl", "rb") as f:
    example_ad = pickle.load(f)

In [6]:
data = "ALS"

f = open('/data_slow/je30bery/spatial_proteomics/config.json')
config = json.load(f)
data_path = "/data_slow/je30bery/data/ALS"  #config[data]
seg = MELC_Segmentation(data_path, membrane_markers="cd45") 
# membrane_marker: str/None 
# radius: multiple of cell radius

In [7]:
for segment in ["nuclei", "cell"]:
    comorbidities = pd.read_csv("/data_slow/je30bery/data/ALS/ALS_comorbidities.txt", delimiter=";")
    comorbidities = comorbidities.set_index("pat_id")

    EA = ExpressionAnalyzer()
    EA.run(segment=segment, profile=None)
    expression_data = EA.expression_data.sort_index()

    for fov in seg.fields_of_view:
        if "ipynb" in fov:
            continue

        seg.field_of_view = fov
        if os.path.exists(f"/data_slow/je30bery/spatial_proteomics/segmentation_results/{fov}_nuclei.pickle"):
            with open(f"/data_slow/je30bery/spatial_proteomics/segmentation_results/{fov}_nuclei.pickle", "rb") as handle:
                where_nuc = pickle.load(handle)
            with open(f"/data_slow/je30bery/spatial_proteomics/segmentation_results/{fov}_cell.pickle", "rb") as handle:
                where_cell = pickle.load(handle)
            nuc = np.load(f"/data_slow/je30bery/spatial_proteomics/segmentation_results/{fov}_nuclei.npy")
            cell = np.load(f"/data_slow/je30bery/spatial_proteomics/segmentation_results/{fov}_cells.npy")
        else:
            nuc, mem, where_nuc, where_mem = seg.run()

        where_dict = where_nuc if segment == "nuclei" else where_cell   
        where_dict = dict(sorted(where_dict.items()))
        
        markers = {
            m.split("_")[1]: os.path.join(seg.get_fov_dir(), m)
            for m in sorted(os.listdir(seg.get_fov_dir()))
            if m.endswith(".tif") and "phase" not in m
        }
            
        adata = ad.AnnData(expression_data.loc[fov].iloc[:,:-2])
        adata.obsm["cell_id"] = expression_data.loc[fov].index.astype(int).to_numpy()
        adata.obsm["segment_size"] = np.array([len(where_dict[k][0]) for k in where_dict])
        
        adata.obsm["group"] = expression_data.loc[fov]["Group"].astype(str).values
        adata.obsm["patient_id"] = expression_data.loc[fov]["Sample"].astype(str).values
        
        adata.uns["group"] = np.unique(expression_data.loc[fov]["Group"].astype(str).values)[0]
        adata.uns["patient_id"] = np.unique(expression_data.loc[fov]["Sample"].astype(str).values)[0]
        
        sample = np.unique(expression_data.loc[fov]["Sample"])[0]
        for c in comorbidities.columns:
            if "ALS" in sample:
                adata.obsm[str(c)] = np.array([str(comorbidities.loc[sample, c])]* expression_data.loc[fov].shape[0])
                adata.uns[str(c)] = str(comorbidities.loc[sample, c])
            else:
                adata.obsm[str(c)] = np.array(["unknown"] * expression_data.loc[fov].shape[0])
                adata.uns[str(c)] = "unknown"

        adata.obsm["field_of_view"] = np.array([fov] * expression_data.loc[fov].shape[0]) 
        adata.uns["field_of_view"] = fov
        
    
        if "spatial" not in adata.uns:
            adata.uns["spatial"] = {}  # Create the "spatial" key if it doesn't exist

        if "images" not in adata.uns["spatial"]:
            adata.uns["spatial"]["images"] = {}  # Create the "images" key if it doesn't exist
        
        adata.uns["spatial"]["images"]["Propidium iodide"] = seg.get_prop_iodide()
        for m in markers:
            adata.uns["spatial"]["images"][str(m)] = cv2.imread(markers[m], cv2.IMREAD_GRAYSCALE)
    
        adata.obsm["control_mean_expression"] = np.array([EA.expression_data[EA.expression_data["Group"] == "Control"].iloc[:, :-2].mean(axis=0).values] * expression_data.loc[fov].shape[0])
        adata.obsm["control_std_expression"] = np.array([EA.expression_data[EA.expression_data["Group"] == "Control"].iloc[:, :-2].std(axis=0).values] * expression_data.loc[fov].shape[0])       
        adata.uns["control_mean_expression"] = EA.expression_data[EA.expression_data["Group"] == "Control"].iloc[:, :-2].mean(axis=0).values
        adata.uns["control_std_expression"] = EA.expression_data[EA.expression_data["Group"] == "Control"].iloc[:, :-2].std(axis=0).values


    
    
        adata.uns["coordinates"] = where_dict
        adata.uns["spatial"]["segmentation"] = nuc if segment == "nuclei" else cell
        
        print(fov)        
        with open(f"/data_slow/je30bery/spatial_proteomics/ann_data/{segment}_{fov}.pickle", 'wb') as handle:
            pickle.dump(adata, handle, protocol=pickle.HIGHEST_PROTOCOL)
        break

Calculating expression: 100%|███████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 458.06it/s]
anndata.py (117): Transforming to str index.


ALS01 - 21297


Calculating expression: 100%|███████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 398.69it/s]
anndata.py (117): Transforming to str index.


ALS01 - 21297
